<a href="https://colab.research.google.com/github/joew2k/WQU_ml_fin/blob/main/Working_with_Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The keras Tunner is a library that helps you pick the optimal set of hyper-parameters for your TensorFlow program

There are two types of hyperparameters:
- *Model Hyperparameters:* Which influence model selection such as the number and width of hidden layers
- *Algorithm hyperparameters:* This influence the speed and qualityy of the learning algorithm such as learning rate for SGD and number of neighbors for a KNN classifier

In [5]:
import tensorflow as tf
from tensorflow import keras
try:
    import keras_tuner as kt
except:
    !pip install keras-tuner
    import keras_tuner as kt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.4 MB/s eta 0:00:00


In [2]:
# Download and Prepare the Dataset
# We will be using the Fashion MNIST dataset
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [3]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') /  255.0
img_test = img_test.astype('float32') / 255.0

## Define the model
When building a model for hypertuning, there is need to define the hyperparameter search space in addition to the model architecture. The model setup for hypertuning is call a hypermodel. This can be archieved in two ways
1. Model builder function
2. Subclassing the *HyperModel* class of the keras tuner API

This program will be working with model bulder function to define the image classification model. The model builder function returns a compiled model and uses hyperparameters you define inline to hypertune the model.

In [4]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(10))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate), loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

    return model

## Instantiate the tuner and perform hypertuning
Instantiate the tuner to perform the hypertuning. The keras tuner has four tuners available *RandomSearch, Hyperband, BayesianOptimization* and *Sklearn*. **Hyperband** tuner will be used here

In [7]:
tuner = kt.Hyperband(
    model_builder,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='my_dir',
    project_name ='intro_to_kt'
)

The Hyperband tuning algorithm uses adaptive resource allocation and early-stopping to quickly converge on a high-performing model. This is done using a sports championship style bracket. The algorithm trains a larger number of models for a few epochs and carries forward only the top performing half of models to the next round. Hyperband determines the number of models to train in a bracket by computing $1 + log_{factor}(max_epochs)$ and rounding it up to the nearest integer

Create a callback to stop training early after reaching a certain value for the validation loss

In [8]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [10]:
# Run the hyperparameter search. the arguments for the search method are the same as those used for t
# tf.keras.model.fit in addition to the callback above
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f'''The hyperparameter search is complete. The optimal number
 of units in the first densensly-connected layer is {best_hps.get('learning_rate')}''')

The hyperparameter search is complete. The optimal number
 of units in the first densensly-connected layer is 0.001


## Train the model
Find the optimal number of epochs to train the model with the hyperparameters obtained from the search

In [11]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 10s 6ms/step - loss: 0.5009 - accuracy: 0.8230 - val_loss: 0.4146 - val_accuracy: 0.8545
Epoch 2/50
1500/1500 [==============================] - 10s 7ms/step - loss: 0.3740 - accuracy: 0.8635 - val_loss: 0.3782 - val_accuracy: 0.8639
Epoch 3/50
1500/1500 [==============================] - 10s 7ms/step - loss: 0.3341 - accuracy: 0.8761 - val_loss: 0.3437 - val_accuracy: 0.8794
Epoch 4/50
1500/1500 [==============================] - 10s 7ms/step - loss: 0.3085 - accuracy: 0.8869 - val_loss: 0.3314 - val_accuracy: 0.8787
Epoch 5/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2925 - accuracy: 0.8919 - val_loss: 0.3541 - val_accuracy: 0.8717
Epoch 6/50
1500/1500 [==============================] - 10s 7ms/step - loss: 0.2738 - accuracy: 0.8993 - val_loss: 0.3468 - val_accuracy: 0.8796
Epoch 7/50
1500/1500 [==============================] - 10s 7ms/step - loss: 0.2601 - accuracy: 0.9021 - val_loss: 0.3146 - val_acc

Re-instantiate the hypermodel and train it with the optimal number of epochs from above

In [12]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/43
1500/1500 [==============================] - 11s 7ms/step - loss: 0.4972 - accuracy: 0.8244 - val_loss: 0.4243 - val_accuracy: 0.8444
Epoch 2/43
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3706 - accuracy: 0.8655 - val_loss: 0.3621 - val_accuracy: 0.8668
Epoch 3/43
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3314 - accuracy: 0.8790 - val_loss: 0.3632 - val_accuracy: 0.8654
Epoch 4/43
1500/1500 [==============================] - 11s 7ms/step - loss: 0.3082 - accuracy: 0.8855 - val_loss: 0.3409 - val_accuracy: 0.8748
Epoch 5/43
1500/1500 [==============================] - 10s 6ms/step - loss: 0.2868 - accuracy: 0.8938 - val_loss: 0.3354 - val_accuracy: 0.8783
Epoch 6/43
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2724 - accuracy: 0.8997 - val_loss: 0.3252 - val_accuracy: 0.8814
Epoch 7/43
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2576 - accuracy: 0.9032 - val_loss: 0.3321 - val_accura

In [13]:
# Evaluate the model on the test data
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 3ms/step - loss: 0.5517 - accuracy: 0.8907
[test loss, test accuracy]: [0.5517007112503052, 0.8906999826431274]
